### Modeling of manually-derived strains
Computer-aided cell factory engineering was quite recently introduced to metabolic engineering, as a high throuput analysis tool to identify gene modifications and simulate the chaotic metabolic processes incide the cells. However, metabolic engineering has a longer history, and there have been a lot of efforts to achieve high production of desired products, by lowering or increasing the expression of intuitive and logically derived gene targets.
In this section, we are going to evaluate some popular manual derived changes through model simulations. Those are the downregulation of squalene synthase (<i>ERG9</i>), squalene epoxidase (<i>ERG1</i>), and sterol C-5 desaturase (<i>ERG3</i>) the upregulation of 3-hydroxy-3-methylglutaryl-coenzyme A reductase (<i>HMG1</i>) and the deletion of aldehyde dehydrogenase (<i>ALD6</i>).

Loading the model

In [1]:
from cobra.io import read_sbml_model, write_sbml_model
model = read_sbml_model('/Users/nikow/github-classroom/27410/group-assingment-team-4/yeast-GEM-8.7.0/model/yeast-GEM2.xml') # loading
from cobra import Reaction, Metabolite, Gene
import pandas as pd

We will locate the reactions in the model

In [4]:
model.reactions.query('squalene synthase', 'name')

[<Reaction r_1012 at 0x159ffcccf50>]

In [5]:
model.reactions.r_1012

Reaction identifier,r_1012
Name,squalene synthase
Memory address,0x159ffcccf50
Stoichiometry,2.0 s_0190 + s_0794 + s_1212 --> 2.0 s_0633 + s_1207 + s_1447 2.0 farnesyl diphosphate + H+ + NADPH --> 2.0 diphosphate + NADP(+) + squalene
GPR,YHR190W
Lower bound,0.0
Upper bound,1000.0


In [6]:
model.reactions.query('squalene epoxidase', 'name')

[<Reaction r_1010 at 0x159ffdf9a90>, <Reaction r_1011 at 0x159ffe68290>]

In [7]:
model.reactions.r_1010

Reaction identifier,r_1010
Name,squalene epoxidase (NAD)
Memory address,0x159ffdf9a90
Stoichiometry,"s_0795 + s_1204 + s_1276 + s_1448 --> s_0038 + s_0804 + s_1199 H+ + NADH + oxygen + squalene --> (S)-2,3-epoxysqualene + H2O + NAD"
GPR,YGR175C
Lower bound,0.0
Upper bound,1000.0


In [12]:
model.reactions.query('sterol desaturase', 'name')

[<Reaction r_0233 at 0x159ff9c7090>, <Reaction r_0242 at 0x159ff975590>]

In [13]:
model.reactions.r_0242

Reaction identifier,r_0242
Name,C-5 sterol desaturase
Memory address,0x159ff975590
Stoichiometry,"s_0657 + s_0794 + s_1212 + s_1275 --> s_0664 + 2.0 s_0803 + s_1207 episterol + H+ + NADPH + oxygen --> ergosta-5,7,24(28)-trien-3beta-ol + 2.0 H2O + NADP(+)"
GPR,YLR056W
Lower bound,0.0
Upper bound,1000.0


In [56]:
model.reactions.query('hydroxymethylglutaryl CoA reductase', 'name')

[<Reaction r_0558 at 0x159ffbba390>]

In [55]:
model.reactions.r_0558

Reaction identifier,r_0558
Name,hydroxymethylglutaryl CoA reductase
Memory address,0x159ffbba390
Stoichiometry,s_0218 + 2.0 s_0794 + 2.0 s_1212 --> s_0028 + s_0529 + 2.0 s_1207 3-hydroxy-3-methylglutaryl-CoA + 2.0 H+ + 2.0 NADPH --> (R)-mevalonate + coenzyme A + 2.0 NADP(+)
GPR,YLR450W or YML075C
Lower bound,0.0
Upper bound,1000.0


<i>S. cerevisiae</i> posseses two genes encoding 3-hydroxy-3-methylglutaryl-coenzyme A reductase enzymes, <i>HMG1</i> and <i>HMG2</i>. As the isoenzyme HMG1 appears to be the more stable version of the two, it is the one picked usually for upregulation.

In [61]:
model.reactions.query('YPL061W' , 'gene_name_reaction_rule')

[<Reaction r_0173 at 0x159ff706f10>,
 <Reaction r_0177 at 0x159ff933610>,
 <Reaction r_4718 at 0x15990f96c50>,
 <Reaction r_4719 at 0x15990f97610>]

In [72]:
model.reactions.r_0173

Reaction identifier,r_0173
Name,"aldehyde dehydrogenase (acetaldehyde, NADP)"
Memory address,0x159ff706f10
Stoichiometry,s_0359 + s_0803 + s_1207 --> s_0362 + 2.0 s_0794 + s_1212 acetaldehyde + H2O + NADP(+) --> acetate + 2.0 H+ + NADPH
GPR,YPL061W
Lower bound,0.0
Upper bound,1000.0


In [73]:
model.reactions.r_0177

Reaction identifier,r_0177
Name,"aldehyde dehydrogenase (indole-3-acetaldehyde, NADP)"
Memory address,0x159ff933610
Stoichiometry,s_0803 + s_0850 + s_1207 --> 2.0 s_0794 + s_0853 + s_1212 H2O + indol-3-ylacetaldehyde + NADP(+) --> 2.0 H+ + indole-3-acetate + NADPH
GPR,YPL061W
Lower bound,0.0
Upper bound,1000.0


In [74]:
model.reactions.r_4718

Reaction identifier,r_4718
Name,"isobutyraldehyde dehydrogenase, c"
Memory address,0x15990f96c50
Stoichiometry,s_0803 + s_0937 + s_1198 <=> 2.0 s_0794 + s_1203 + s_4285 H2O + isobutyraldehyde + NAD <=> 2.0 H+ + NADH + isobutyrate
GPR,YOR374W or YPL061W
Lower bound,-1000.0
Upper bound,1000.0


In [75]:
model.reactions.r_4719

Reaction identifier,r_4719
Name,"2-methylbutanal dehydrogenase, c"
Memory address,0x15990f97610
Stoichiometry,s_0166 + s_0803 + s_1198 <=> 2.0 s_0794 + s_1203 + s_4282 2-methylbutanal + H2O + NAD <=> 2.0 H+ + NADH + 2-methylbutyrate
GPR,YOR374W or YPL061W
Lower bound,-1000.0
Upper bound,1000.0


In [76]:
model.reactions.query('YOR374W' , 'gene_name_reaction_rule')

[<Reaction r_0174 at 0x159ff947c90>,
 <Reaction r_0175 at 0x159ff86f010>,
 <Reaction r_0176 at 0x159ff882b10>,
 <Reaction r_0178 at 0x159ff951c10>,
 <Reaction r_0201 at 0x159ff8fdcd0>,
 <Reaction r_2116 at 0x1598ffebe50>,
 <Reaction r_4678 at 0x15990f703d0>,
 <Reaction r_4718 at 0x15990f96c50>,
 <Reaction r_4719 at 0x15990f97610>]

It appears that ALD6 catalyzes 4 reactions in total, for two of them being the only enzyme doing so. For the later two, another aldehyde dehydrogenase, ALD4, has the abillity to catalyse the same reactions. As a conclusiom, to demosntrate the deletion of <i>ALD6</i>, we will delete reactions r_0173 and r_0177, as they are the main reactions our enzyme is involved into.